# OpenStreetMap Data Wrangling Project

### Data cleaning script

* ** Program: ** Data Analyst Nanodegree 
* ** Student: ** Guillermo Naranjo
* ** Date:** August 14, 2017

### Objective
Based on data audit, this script apply three processes to clean some issues related to data:
* **unify_postcodes** updates nodes and ways tags table to unify postalcode keys and values.
* **Pandas phone number update ** script will look for problematic phone numbers and will fix them.
* **Name uniformity** script will map expected names, updating those unexpected with right values. 

In [1]:
import sqlite3
import re
import pandas as pd

In [3]:
# ================================================== #
#     Global constants are defined                   #
# ================================================== #
UQUERYNODES = '''UPDATE nodes_tags SET key = 'postcode', type = 'addr' WHERE key='postal_code';'''
UQUERYWAYS = '''UPDATE ways_tags SET key = 'postcode', type = 'addr' WHERE key='postal_code';'''
SQUERYNODES = '''SELECT * from nodes_tags where key = 'postal_code';'''
SQUERYWAYS = '''SELECT * from ways_tags where key = 'postal_code';'''
SQUERYNODETAGS = '''SELECT * from nodes_tags;'''
SQUERYWAYTAGS = '''SELECT * from ways_tags;'''
RESULT = {True: 'Successful',False: 'Failed'}
DBFILE = 'osm_costa_rica.db'
VALID_PHONE = re.compile(r'\(\d{3}\)\s*\d{8}') #using https://pythex.org/

MAPPING = {
    "AM-PM":"AMPM",
    "Aeroscasillas":"Aerocasillas",
    "Azafran":"Azafrán",
    "Cerrajeria":"Cerrajería",
    "Coopeserviodores":"Coopeservidores",
    "JetBox":"Jetbox",
    "Macrobiotica":"Macrobiótica",
    "MegaSuper":"Megasuper",
    "Metropolis":"Metrópolis",
    "Metropoli":"Metrópolis",
    "Musammani":"Musmanni",
    "Musmani":"Musmanni",
    "Musmanny":"Musmanni",
    "Mussmani":"Musmanni",
    "Muswanni":"Musmanni",
    "Panaderia":"Panadería",
    "Pizzeria":"Pizzería",
    "Pulperia":"Pulpería",
    "Quiros":"Quirós",
    "Veterniaria":"Veterinaria",
    "Cartago1":"Cartago",
    "Rohmoser":"Rohrmoser"
}

### 2. This algoritm unify postcodes with basic python libraries

In [37]:
# ================================================== #
#               PostCode Cleaning Functions         #
# ================================================== #
def create_connection(db_file):
    """ create a database connection to the SQLite database specified by the db_file """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
    return None

def update(script, db_file=DBFILE):
    """ This script runs a given sql update query """
    conn = create_connection(db_file)
    cur = conn.cursor()
    cur.execute(script)
    conn.commit()
    conn.close()
    
def exists(script, db_file=DBFILE):
    """ This script runs a given sql select query and return true is has more than 1 row, false otherwise"""
    conn = create_connection(db_file)
    cur = conn.cursor()        
    cur.execute(script)
    row = cur.fetchone()
    conn.commit()
    conn.close()
    return False if row is None else True

def unify_postcodes(query_nodes=UQUERYNODES,query_ways=UQUERYWAYS, squery_nodes=SQUERYNODES, squery_tags = SQUERYWAYS):
    """ This script apply updates on nodes_tags and ways_tags tables to unify post_code keys """                
    update(query_nodes)
    update(query_ways)
    print "1. Postal codes nodes tags unification:", RESULT[not exists(squery_nodes)]
    print "2. Postal codes ways tags unification:", RESULT[not exists(squery_tags)]

In [8]:
if __name__ == '__main__':
    unify_postcodes()

1. Postal codes nodes tags unification: Successful
2. Postal codes ways tags unification: Successful


### 3.Here I want to use pandas to clean phone numbers using Apply function

In [35]:
# ================================================== #
#           Phone Numbers Cleaning Functions         #
# ================================================== #

def is_valid_phone(phone):
    return  VALID_PHONE.search(phone) is not None

def fix_phone(phone):
    # taken from: https://stackoverflow.com/questions/5658369/how-to-input-a-regex-in-string-replace 
    phone = re.sub(r"-|\(|\)|\ *","", phone).replace("tel:","")
    if len(phone) == 11:
        phone = "(506)" + phone[-8:]
    elif len(phone) == 8:
        phone = "(506)" + phone    
    return phone

** Cleaning node tag phone numbers **

In [15]:
''' Connection is opened, dataframes are filled with node tags data and valid phones are filtered'''
cnx = sqlite3.connect(DBFILE)
tags = pd.read_sql_query(SQUERYNODETAGS, cnx)
phones = tags[tags['key'] == 'phone']
is_valid = phones['value'].apply(is_valid_phone)
valid_phones = phones[is_valid]
print '''NODE Phones with valid formats:''', valid_phones['id'].count()
valid_phones.head()

NODE Phones with valid formats: 204


id    key          value       type
431   272884735  phone  (506)25240006  regular\r
533   275730386  phone  (506)22395717  regular\r
961   292783024  phone  (506)22428300  regular\r
1106  344933160  phone  (506)22878700  regular\r
1259  430768293  phone  (506)22546281  regular\r

In [16]:
invalid_phones = phones[is_valid == False]
print '''NODE Phones with invalid format:''', invalid_phones['id'].count()
invalid_phones.head(invalid_phones['id'].count())

NODE Phones with invalid format: 2


id    key          value       type
1305    430992167  phone     9052463722  regular\r
16881  4207664678  phone  5062511556402  regular\r

In [17]:
''' Updates tags dataframe with fixed values for phone types '''
# I'm using this approach: https://stackoverflow.com/questions/14431646/how-to-write-pandas-dataframe-to-sqlite-with-index
tags.loc[tags[tags['key']=='phone'].index,'value'] = tags[tags['key']=='phone']['value'].apply(fix_phone)
tags = tags.set_index('id')
tags = tags[tags.value != 'value']
tags.head()

key              value       type
id                                              
114940878  exit_to           Alajuela  regular\r
114940878  highway  motorway_junction  regular\r
114940955  highway  motorway_junction  regular\r
114940955     name                111  regular\r
115577985  highway    traffic_signals  regular\r

In [18]:
''' we finally write the NODES dataframe content to database, commit and close connection'''
tags.to_sql("nodes_tags", cnx, if_exists="replace")
cnx.commit()
cnx.close()

** Cleaning ways phone tags **

* We use the same procedure above, applied to nodes tags.

In [22]:
''' Connection is opened, dataframes are filled with way tags data and valid phones are filtered'''
cnx = sqlite3.connect(DBFILE)
tags = pd.read_sql_query(SQUERYWAYTAGS, cnx)
phones = tags[tags['key'] == 'phone']

is_valid = phones['value'].apply(is_valid_phone)

valid_phones = phones[is_valid]
print '''WAY Phones with valid formats:''', valid_phones['id'].count()

invalid_phones = phones[is_valid == False]
print '''WAY Phones with invalid format:''', invalid_phones['id'].count()

WAY Phones with valid formats: 893
WAY Phones with invalid format: 13


In [23]:
''' Updates tags dataframe with fixed values for phone types '''
tags.loc[tags[tags['key']=='phone'].index,'value'] = tags[tags['key']=='phone']['value'].apply(fix_phone)
tags = tags.set_index('id')
tags = tags[tags.value != 'value']
tags.head()

key    value       type
id                                   
12569965  aeroway   runway  regular\r
12569965   length     1600  regular\r
12569965      ref    09-27  regular\r
12569965  surface  asphalt  regular\r
12569970  aeroway  taxiway  regular\r

In [24]:
''' we finally write the WAYS dataframe content to database, commit and close connection'''
tags.to_sql("ways_tags", cnx, if_exists="replace")
cnx.commit()
cnx.close()

** Now, there are only 15 records that can be manually updated as required :) **

### Tag Name correction script using pandas (I like pandas)

In [36]:
# ================================================== #
#               Tag Names Cleaning Functions         #
# ================================================== #

def update_name(name, mapping=MAPPING):
    name = name.encode('utf-8')
    my_map = pd.Series(mapping)
    return my_map[name] if name in my_map else name

** Cleaning node tag names first **

In [26]:
cnx = sqlite3.connect(DBFILE)
cnx.text_factory = lambda x: unicode(x, "utf-8", "ignore") # this solved utf-8 issue: http://subforge.org/blogs/show/25
tags = pd.read_sql_query(SQUERYNODETAGS, cnx)
tags.loc[tags[tags['key']=='name'].index,'value'] = tags[tags['key']=='name']['value'].apply(update_name)
tags = tags.set_index('id')
tags = tags[tags.value != 'value']
tags.head()

key              value       type
id                                              
114940878  exit_to           Alajuela  regular\r
114940878  highway  motorway_junction  regular\r
114940955  highway  motorway_junction  regular\r
114940955     name                111  regular\r
115577985  highway    traffic_signals  regular\r

In [27]:
''' we finally write the NODES dataframe content to database, commit and close connection'''
tags.to_sql("nodes_tags", cnx, if_exists="replace")
cnx.commit()
cnx.close()

** Finally, cleaning ways tag names **

In [28]:
cnx = sqlite3.connect(DBFILE)
cnx.text_factory = lambda x: unicode(x, "utf-8", "ignore") # this solved utf-8 issue: http://subforge.org/blogs/show/25
tags = pd.read_sql_query(SQUERYWAYTAGS, cnx)
tags.loc[tags[tags['key']=='name'].index,'value'] = tags[tags['key']=='name']['value'].apply(update_name)
tags = tags.set_index('id')
tags = tags[tags.value != 'value']
tags.head()

key    value       type
id                                   
12569965  aeroway   runway  regular\r
12569965   length     1600  regular\r
12569965      ref    09-27  regular\r
12569965  surface  asphalt  regular\r
12569970  aeroway  taxiway  regular\r

In [29]:
''' we finally write the WAYS dataframe content to database, commit and close connection'''
tags.to_sql("ways_tags", cnx, if_exists="replace")
cnx.commit()
cnx.close()